See [example_agent.ipynb](https://github.com/krasserm/grammar-based-agents/blob/wip-article-2/example_agent.ipynb) for a fully documented version of this notebook based on an [earlier version](https://github.com/krasserm/grammar-based-agents/tree/wip-article-2) of this project. The following code is compatible with the latest version.

In [1]:
from sentence_transformers import SentenceTransformer, CrossEncoder

from gba.client import MistralInstruct, Llama3Instruct, LlamaCppClient


mistral_instruct = MistralInstruct(
    llm=LlamaCppClient(url="http://localhost:8081/completion", temperature=-1),
)

llama3_instruct = Llama3Instruct(
    llm=LlamaCppClient(url="http://localhost:8084/completion", temperature=-1),
)

embedding_model = SentenceTransformer(
    "mixedbread-ai/mxbai-embed-large-v1",
    device="cuda:0",
)

rerank_model = CrossEncoder(
    "mixedbread-ai/mxbai-rerank-large-v1",
    device="cuda:0",
)

searxng_endopoint = "http://localhost:8080"

In [2]:
from gba.agent import Agent
from gba.client import ChatClient
from gba.planner import ZeroShotPlanner
from gba.tools import *
from gba.tools.functions import *


search_wikipedia_tool = SearchWikipediaTool(
    llm=llama3_instruct,
    embedding_model=embedding_model,
    rerank_model=rerank_model,                    
    top_k_nodes=10,
    top_k_related_documents=1,
    top_k_related_nodes=3,
)

search_internet_tool = SearchInternetTool(
    llm=llama3_instruct,
    rerank_model=rerank_model,
    searxng_endpoint=searxng_endopoint,
    fetch_webpage_timeout=5.0,
    top_k_documents=3,
    top_k_nodes_per_document=5,
    top_k_snippets=None,
)

summarizer = ResultSummarizer(model=llama3_instruct)

tools = [
    AskTool(),
    CalculateTool(model=llama3_instruct, summarizer=summarizer),
    FunctionCallTool(model=llama3_instruct, fn=create_event),
    FunctionCallTool(model=llama3_instruct, fn=send_email),
    FunctionCallTool(model=llama3_instruct, fn=search_images),
    search_internet_tool,
    search_wikipedia_tool,    
    RespondTool(model=llama3_instruct),
]

client = ChatClient(model=mistral_instruct)
planner = ZeroShotPlanner(client=client, tools_spec=ToolsSpec(tools))
agent = Agent(planner=planner, tools=tools)

Loading int8 search index...
Loading document mapping...
Loading binary search index...


In [4]:
agent.run("who is Leo DiCaprio's current girlfriend and what is her age raised to the power of 0.24?")

Task: Search the internet to find out who Leo DiCaprio's current girlfriend is.


Searching the internet for query 'Leo DiCaprio current girlfriend'


Observation: According to the context, Vittoria Ceretti is Leonardo DiCaprio's new model girlfriend.

Task: Search the internet to find out the age of Vittoria Ceretti.


Searching the internet for query 'What is Vittoria Ceretti's age?'


Observation: As of 2023, Vittoria Ceretti is 25 years old.

Task: Calculate the value of 25 raised to the power of 0.24.
```python
vittoria_age = 25
result = pow(vittoria_age, 0.24)
```
Observation: 25 raised to the power of 0.24 is 2.165237809112449.



"Leo DiCaprio's current girlfriend is Vittoria Ceretti, who is 25 years old and her age raised to the power of 0.24 is approximately 2.165237809112449."

In [5]:
agent.run("Add Martin's birthday party, starting Dec. 17th 2023 8pm, to my calendar and respond with a one-line invitation")

Task: Creating Martin's birthday party event on Dec. 17th, 2023 at 8 PM and generating a one-line invitation.
Observation: Event Martin's birthday party successfully added to calendar, date=2023-12-17, time=20:00:00



"You're invited to Martin's birthday party on December 17th, 2023 at 8pm!"

In [6]:
agent.run("Tell me a joke")

'Why was the math book sad? Because it had too many problems.'

### Conversational tool use



In [ ]:
conversational_agent = Agent(planner=planner, tools=tools, conversational=True)

In [ ]:
conversational_agent.run("I want an image of a dog")

In [ ]:
conversational_agent.run("It should be brown")

In [ ]:
conversational_agent.run("Find an image with two")